In [175]:
import os
import json
import pandas as pd
import traceback

In [176]:
from langchain.chat_models import ChatOpenAI

In [177]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [178]:
KEY=os.getenv("OPENAI_API_KEY")

In [179]:
KEY='sk-proj-eZdAwGNx4otstQ29gOHhT3BlbkFJGwQgXxovlT3oxR7SHjFp'

In [180]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [181]:
KEY

'sk-proj-eZdAwGNx4otstQ29gOHhT3BlbkFJGwQgXxovlT3oxR7SHjFp'

In [182]:
print(KEY)

sk-proj-eZdAwGNx4otstQ29gOHhT3BlbkFJGwQgXxovlT3oxR7SHjFp


In [183]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000014FB7BB5630>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000014FB7BB72E0>, temperature=0.5, openai_api_key='sk-proj-eZdAwGNx4otstQ29gOHhT3BlbkFJGwQgXxovlT3oxR7SHjFp', openai_proxy='')

In [184]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [185]:
with open(r"C:\Users\Aman\Desktop\genai MCQ Project\MCQ-Generator-Genai-Project\Response.json", "r") as f:
    RESPONSE_JSON=json.load(f)

In [186]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [187]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [188]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    template=TEMPLATE
)

In [189]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

In [190]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [191]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
    )

In [192]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key='review', verbose=True)

In [193]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    output_variables=["quiz", "review"], 
    verbose=True,
    )

In [194]:
PATH = r"C:\Users\Aman\Desktop\genai MCQ Project\MCQ-Generator-Genai-Project\data.txt"

In [195]:
print(PATH)

C:\Users\Aman\Desktop\genai MCQ Project\MCQ-Generator-Genai-Project\data.txt


In [196]:
with open(PATH, "r") as file:
    TEXT=file.read()    

In [197]:
TEXT

'Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to: \n\n\nSystems, architecture and infrastructure for Gen AI\nFoundation models and their applications\nMulti-modal Gen AI \nRetrieval Augmented Generated\nGen AI based plugins and agents\nLearning from human preferences \nEvaluation of f

In [198]:
##MCQ ko generate krwana hai

In [199]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [200]:
import json
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [201]:
NUMBER=5 
SUBJECT="AI"
TONE="Simple"
RESPONSE_JSON=RESPONSE_JSON

In [202]:
response_json_str=json.dumps(RESPONSE_JSON)

In [203]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [204]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "RESPONSE_JSON": response_json_str
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to: 


Systems, architecture and infrastructure for Gen AI
Foundation models and their applications
Multi-modal Gen AI 
Retrieval Au

In [205]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1444
Prompt Tokens:978
Completion Tokens:466
Total Cost:0.002399


In [135]:
response

{'text': 'Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to: \n\n\nSystems, architecture and infrastructure for Gen AI\nFoundation models and their applications\nMulti-modal Gen AI \nRetrieval Augmented Generated\nGen AI based plugins and agents\nLearning from human preferences \nEvalua

In [207]:
quiz=response.get("quiz")

In [210]:
quiz= json.loads(quiz)

In [211]:
quiz

{'1': {'mcq': 'What is the focus of the workshop on Generative AI at AIMLSystems?',
  'options': {'a': 'Building robots',
   'b': 'Developing self-driving cars',
   'c': 'Building and deploying Generative AI systems',
   'd': 'Creating virtual reality games'},
  'correct': 'c'},
 '2': {'mcq': 'Which of the following is NOT a topic of interest for Generative AI discussed in the text?',
  'options': {'a': 'Foundation models and their applications',
   'b': 'Responsible Gen AI',
   'c': 'Developing mobile applications',
   'd': 'ML Ops for Gen AI'},
  'correct': 'c'},
 '3': {'mcq': 'What does Gen AI stand for in the text?',
  'options': {'a': 'General Artificial Intelligence',
   'b': 'Generation Artificial Intelligence',
   'c': 'Genius Artificial Intelligence',
   'd': 'Generating Artificial Intelligence'},
  'correct': 'b'},
 '4': {'mcq': 'Which aspect of Generative AI covers concerns related to fairness, transparency, and accountability?',
  'options': {'a': 'Learning from human prefe

In [212]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join( 
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [214]:
quiz_table_data

[{'MCQ': 'What is the focus of the workshop on Generative AI at AIMLSystems?',
  'Choices': 'a: Building robots | b: Developing self-driving cars | c: Building and deploying Generative AI systems | d: Creating virtual reality games',
  'Correct': 'c'},
 {'MCQ': 'Which of the following is NOT a topic of interest for Generative AI discussed in the text?',
  'Choices': 'a: Foundation models and their applications | b: Responsible Gen AI | c: Developing mobile applications | d: ML Ops for Gen AI',
  'Correct': 'c'},
 {'MCQ': 'What does Gen AI stand for in the text?',
  'Choices': 'a: General Artificial Intelligence | b: Generation Artificial Intelligence | c: Genius Artificial Intelligence | d: Generating Artificial Intelligence',
  'Correct': 'b'},
 {'MCQ': 'Which aspect of Generative AI covers concerns related to fairness, transparency, and accountability?',
  'Choices': 'a: Learning from human preferences | b: Evaluation of foundation models | c: Responsible Gen AI | d: Large language m

In [217]:
df=pd.DataFrame(quiz_table_data)

In [218]:
df

,MCQ,Choices,Correct
0,What is the focus of the workshop on Generativ...,a: Building robots | b: Developing self-drivin...,c
1,Which of the following is NOT a topic of inter...,a: Foundation models and their applications | ...,c
2,What does Gen AI stand for in the text?,a: General Artificial Intelligence | b: Genera...,b
3,Which aspect of Generative AI covers concerns ...,a: Learning from human preferences | b: Evalua...,c
4,What kind of models have resulted in realistic...,a: Predictive models | b: Generative models | ...,b


In [220]:
df.to_csv("genai.csv", index=False)